# Hyperlane Warp Route Deployment Analysis - Sepolia, Pruv & Arbitrum Sepolia Networks

This notebook analyzes the gas costs and transaction patterns for Hyperlane warp route deployment on Sepolia, Pruv, and Arbitrum Sepolia networks.

In [1]:
import json
import pandas as pd
import re
from datetime import datetime
from IPython.display import display

# Define chains for processing
chains = ['arbitrumsepolia', 'pruv', 'sepolia']

## 1. Load and Parse Data

In [2]:
# Load transaction data for all chains
chain_data = {}
chain_dfs = {}

for chain in chains:
    print(f'=== LOADING {chain.upper()} DATA ===')
    with open(f'data/warp_deployment/raw/{chain}_txs.json', 'r') as f:
        chain_data[chain] = json.load(f)
    
    print(f'Total transactions in dataset: {len(chain_data[chain]["result"])}')
    
    # Convert to DataFrame
    chain_dfs[chain] = pd.DataFrame(chain_data[chain]['result'])
    
    # Display basic info
    print(f'DataFrame shape: {chain_dfs[chain].shape}')
    print(f'Columns: {list(chain_dfs[chain].columns)}')
    print()

# Create individual variables for backward compatibility
sepolia_data = chain_data['sepolia']
sepolia_df = chain_dfs['sepolia']
pruv_data = chain_data['pruv']
pruv_df = chain_dfs['pruv']
arbitrumsepolia_data = chain_data['arbitrumsepolia']
arbitrumsepolia_df = chain_dfs['arbitrumsepolia']

=== LOADING ARBITRUMSEPOLIA DATA ===
Total transactions in dataset: 131
DataFrame shape: (131, 18)
Columns: ['blockHash', 'blockNumber', 'confirmations', 'contractAddress', 'cumulativeGasUsed', 'from', 'gas', 'gasPrice', 'gasUsed', 'hash', 'input', 'isError', 'nonce', 'timeStamp', 'to', 'transactionIndex', 'txreceipt_status', 'value']

=== LOADING PRUV DATA ===
Total transactions in dataset: 519
DataFrame shape: (519, 18)
Columns: ['blockHash', 'blockNumber', 'confirmations', 'contractAddress', 'cumulativeGasUsed', 'from', 'gas', 'gasPrice', 'gasUsed', 'hash', 'input', 'isError', 'nonce', 'timeStamp', 'to', 'transactionIndex', 'txreceipt_status', 'value']

=== LOADING SEPOLIA DATA ===
Total transactions in dataset: 1033
DataFrame shape: (1033, 18)
Columns: ['blockHash', 'blockNumber', 'confirmations', 'contractAddress', 'cumulativeGasUsed', 'from', 'gas', 'gasPrice', 'gasUsed', 'hash', 'input', 'isError', 'nonce', 'timeStamp', 'to', 'transactionIndex', 'txreceipt_status', 'value']



## 2. Extract Deployment Start Timestamp

In [3]:
# Extract deployment timestamp from the combined CLI output
print('=== WARP DEPLOYMENT TIMESTAMP ===')
with open('data/warp_deployment/raw/warp_deployment_cli_output.txt', 'r') as f:
    cli_output = f.read()

# Extract timestamp using regex
timestamp_match = re.search(r'"time":(\d{13})', cli_output)

if timestamp_match:
    deployment_timestamp_ms = int(timestamp_match.group(1))
    deployment_timestamp = deployment_timestamp_ms / 1000
    deployment_date = datetime.fromtimestamp(deployment_timestamp)
    print(f'Deployment start timestamp: {deployment_timestamp_ms}')
    print(f'Deployment start date: {deployment_date}')
else:
    print('Timestamp not found in CLI output')
    deployment_timestamp = None

=== WARP DEPLOYMENT TIMESTAMP ===
Deployment start timestamp: 1754972649686
Deployment start date: 2025-08-12 11:24:09.686000


## 3. Filter and Process Transactions

In [4]:
# Process transactions for all chains
deployment_txs = {}

for chain in chains:
    print(f'=== {chain.upper()} TRANSACTION FILTERING ===')
    
    # Get the dataframe for this chain
    df = chain_dfs[chain]
    
    # Convert timestamp and datetime columns
    df['timeStamp'] = pd.to_numeric(df['timeStamp'])
    df['datetime'] = pd.to_datetime(df['timeStamp'], unit='s')
    
    # Filter transactions from deployment start
    if deployment_timestamp:
        deployment_txs[chain] = df[df['timeStamp'] >= deployment_timestamp].copy()
    else:
        # If no timestamp found, use all transactions
        deployment_txs[chain] = df.copy()
    
    # Convert gas and value columns to numeric
    deployment_txs[chain]['gasUsed'] = pd.to_numeric(deployment_txs[chain]['gasUsed'])
    deployment_txs[chain]['gasPrice'] = pd.to_numeric(deployment_txs[chain]['gasPrice'])
    deployment_txs[chain]['value'] = pd.to_numeric(deployment_txs[chain]['value'])
    
    # Calculate gas cost in ETH
    deployment_txs[chain]['gasCostWei'] = deployment_txs[chain]['gasUsed'] * deployment_txs[chain]['gasPrice']
    deployment_txs[chain]['gasCostETH'] = deployment_txs[chain]['gasCostWei'] / 1e18
    
    # Identify transaction types
    deployment_txs[chain]['isContractDeployment'] = deployment_txs[chain]['to'].isna() | (deployment_txs[chain]['to'] == '')
    
    # Print statistics
    print(f'Deployment transactions: {len(deployment_txs[chain])}')
    
    if len(deployment_txs[chain]) > 0:
        print(f'\nFirst transaction: {deployment_txs[chain]["datetime"].min()}')
        print(f'Last transaction: {deployment_txs[chain]["datetime"].max()}')
    
    print()

# Create individual variables for backward compatibility
sepolia_deployment_txs = deployment_txs['sepolia']
pruv_deployment_txs = deployment_txs['pruv']
arbitrumsepolia_deployment_txs = deployment_txs['arbitrumsepolia']

=== ARBITRUMSEPOLIA TRANSACTION FILTERING ===
Deployment transactions: 8

First transaction: 2025-08-12 04:24:15
Last transaction: 2025-08-12 04:26:52

=== PRUV TRANSACTION FILTERING ===
Deployment transactions: 8

First transaction: 2025-08-12 04:24:15
Last transaction: 2025-08-12 04:26:33

=== SEPOLIA TRANSACTION FILTERING ===
Deployment transactions: 8

First transaction: 2025-08-12 04:24:24
Last transaction: 2025-08-12 04:26:48



## 4. Detailed Transaction Tables

In [5]:
# Create detailed transaction tables for all chains
for chain in chains:
    print(f'=== {chain.upper()} DETAILED TRANSACTION TABLE ===')
    print()
    
    txs = deployment_txs[chain]
    
    if len(txs) > 0:
        # Create detailed table
        detailed_table = pd.DataFrame({
            'Timestamp': txs['datetime'],
            'TX Hash': txs['hash'],
            'Context': txs['isContractDeployment'].apply(lambda x: 'Contract Deployment' if x else 'Contract Call'),
            'Gas Limit': pd.to_numeric(txs['gas']),
            'Gas Used': txs['gasUsed'],
            'Gas Price (Wei)': txs['gasPrice'],
            'TX Fee (ETH)': txs['gasCostETH'].round(8)
        })
        
        display(detailed_table)
    else:
        print('No transactions found')
    
    print()

=== ARBITRUMSEPOLIA DETAILED TRANSACTION TABLE ===



,Timestamp,TX Hash,Context,Gas Limit,Gas Used,Gas Price (Wei),TX Fee (ETH)
0,2025-08-12 04:26:52,0xc5aca921fd37030a91d98d7e234bda0506b14267e715...,Contract Call,39567,35693,100000000,0.000004
1,2025-08-12 04:26:30,0xb0c9573844ad5ecb0f094c0ec56d1447a91e912b500d...,Contract Call,80905,79662,100000000,0.000008
2,2025-08-12 04:26:07,0x21c5062c5c69ea2fad20cc4a4cc3763e2e92dfb8b464...,Contract Call,212061,189819,100000000,0.000019
3,2025-08-12 04:25:53,0x81ea1d1ee643c96dc35efad321dcca9501a8ff44ab61...,Contract Deployment,917677,827786,100000000,0.000083
4,2025-08-12 04:25:38,0xfda6bd95dd427f1ed90cfc4d6e80d4f9427b44d95510...,Contract Call,94656,84729,100000000,0.000008
5,2025-08-12 04:25:35,0x2932896ff7032fbda72ea035a7bdf11b5ad40fba32ba...,Contract Deployment,2722436,2455757,100000000,0.000246
6,2025-08-12 04:24:17,0xac56e7fe7824616d1f1fb4cfe2c072d6011b39b88008...,Contract Call,56869,55568,100000000,0.000006
7,2025-08-12 04:24:15,0x9e57c1cc56227caf8c2c4832d2d6044943e1ecc7c98f...,Contract Deployment,1751406,1579848,100000000,0.000158



=== PRUV DETAILED TRANSACTION TABLE ===



,Timestamp,TX Hash,Context,Gas Limit,Gas Used,Gas Price (Wei),TX Fee (ETH)
0,2025-08-12 04:26:33,0x3f470eb0e88ec7424213e31c808f9ca4e257dbd0d83b...,Contract Call,39344,35693,0,0.0
1,2025-08-12 04:26:15,0x90f1775e5056d3b9641ebd6e9e7aa0d1606dd3d37c1f...,Contract Call,80438,79674,0,0.0
2,2025-08-12 04:26:07,0x06ef56140c43f2ca5674370138e910c3b5e242ccbc8f...,Contract Call,211565,189831,0,0.0
3,2025-08-12 04:24:51,0x9f2de642f0a09704ad3770ee520c1b570d60e8cc2014...,Contract Deployment,910261,827510,0,0.0
4,2025-08-12 04:24:47,0x69b1a07e11b260817fff5ec6022e523ab2ff7d15535d...,Contract Call,94448,84729,0,0.0
5,2025-08-12 04:24:43,0x0c05f862abf582a4ed2283d62e58049209971ab1180b...,Contract Deployment,2722802,2475275,0,0.0
6,2025-08-12 04:24:19,0xe03b1f281c5466f05ac816b9052a412a102dd99ae63d...,Contract Call,56690,55568,0,0.0
7,2025-08-12 04:24:15,0x3e189ce02396edb9c71d021003018a8e3f5eac5e8820...,Contract Deployment,1737316,1579379,0,0.0



=== SEPOLIA DETAILED TRANSACTION TABLE ===



,Timestamp,TX Hash,Context,Gas Limit,Gas Used,Gas Price (Wei),TX Fee (ETH)
0,2025-08-12 04:26:48,0x5bd7b56e5ea83da62513df7452ada489d866fe48679f...,Contract Call,39660,35693,1009573,4.000000e-08
1,2025-08-12 04:26:24,0x4765af029f844b89823d3353e066afc242387921e28a...,Contract Call,80561,79662,1011127,8.000000e-08
2,2025-08-12 04:26:12,0x295734c875c518f5ce80a6b1f87f1873a23ebd8d644a...,Contract Call,211742,189819,1011976,1.900000e-07
3,2025-08-12 04:25:24,0x7448b7d7cb3d64da2751d06224dbebd4394cc19b8bdd...,Contract Deployment,919088,827798,1013011,8.400000e-07
4,2025-08-12 04:25:12,0x0ee80a60df91fb4559afd3f714ab48541e4763042f32...,Contract Call,95225,84729,1014236,9.000000e-08
5,2025-08-12 04:25:00,0x9cbab903f59113ab0d168ad35f8e853150d05806d1d0...,Contract Deployment,2724054,2455756,1012673,2.490000e-06
6,2025-08-12 04:24:36,0xa739b0084202e566ca92ffe7e65a80116ac6f92e4297...,Contract Call,57176,55568,1014030,6.000000e-08
7,2025-08-12 04:24:24,0xa560fad614b28c0071d6a2be1791fc38f54ad70f1f1d...,Contract Deployment,1752908,1579847,1014688,1.600000e-06


## 5. Gas Usage Summary

In [6]:
# Calculate gas usage summary for all chains
print('=== WARP DEPLOYMENT GAS USAGE SUMMARY ===')
print()

summary_data = []

for chain in chains:
    txs = deployment_txs[chain]
    
    total_gas_used = txs['gasUsed'].sum()
    total_gas_limit = pd.to_numeric(txs['gas']).sum()
    total_cost_eth = txs['gasCostETH'].sum()
    avg_gas_price = txs['gasPrice'].mean()
    gas_efficiency = (total_gas_used / total_gas_limit) * 100 if total_gas_limit > 0 else 0
    
    summary_data.append({
        'Chain': chain.title(),
        'Total Gas Limit': f'{total_gas_limit:,}',
        'Total Gas Used': f'{total_gas_used:,}',
        
        'Average Gas Price (Wei)': f'{avg_gas_price:,.0f}',
        'Gas Efficiency (%)': f'{gas_efficiency:.2f}%',
        'Total Deployment Cost (ETH)': f'{total_cost_eth:.8f}'
    })

summary_df = pd.DataFrame(summary_data)
display(summary_df)

=== WARP DEPLOYMENT GAS USAGE SUMMARY ===



,Chain,Total Gas Limit,Total Gas Used,Average Gas Price (Wei),Gas Efficiency (%),Total Deployment Cost (ETH)
0,Arbitrumsepolia,"5,875,577","5,308,862","100,000,000",90.35%,0.00053089
1,Pruv,"5,852,864","5,327,659",0,91.03%,0.00000000
2,Sepolia,"5,880,414","5,308,872","1,012,664",90.28%,0.00000538
